In [13]:
# 임포트
# 임포트
import pandas as pd
import numpy as np
import langchain 
import requests
from bs4 import BeautifulSoup

# 서제스트 키워드 긁어오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [14]:
# 임포트 2
from create_URL import createDaumURL
from create_URL import createGoogleURL
from create_URL import createYoutubeURL
from create_URL import createNaverURL

from getOpenKeywords import getOpenKeywords
from keywordFormatting import keywordFormatting
from seleniumClickAndCrawling import seleniumClickAndCrawling
from countDaumADs import countDaumADs
from count_GoogleSearchVolume import countGoogleSearchVolume
from checkGoogleSectionOrder import checkGoogleSectionOrder
from getGoogleOpenKeywords import getGoogleOpenKeywords
from limitLength import limitLength

In [15]:
# config _ format 설정
# config
# 수집데이터
keywords = {}

providers = ['daumOpenKeywords',
'daumSuggestKeywords',
'googleSuggestKeywords',
'googleOpenKeywords',
# 'youtubeSuggestKeywords',
]
benchmarks = ['searchVolume','countDaumADs','tistoryRank','provider']

searchKeywordFormat = {}

# searchKeywordFormat = {
        # 'daumOpenKeywords' : {},
        # 'daumSuggestKeywords' : {},
        # 'naverOpenKeywords' : {},
        # 'googleSuggestKeywords' : {},
        # 'googleOpenKeywords':{},
        # 'youtubeSuggestKeywords':{},
        # 'mergedKeywords' : {},
        # }    

keywordValuationFormat = {
    'searchVolume' : '',
    'countDaumADs' : '',
    'tistoryRank' : '',    
    'provider' : '',
    'depth' : ''  
}


In [16]:
# config _ CSSSelector설정

## 오픈키워드 선택자
# daumOpenKeywordsTag = ['#netizen_lists_top span', '#recomm_lists_top a'] # 관련키워드
daumOpenKeywordsTag =  'a.keyword'# 관련키워드

naverOpenKeywordsTag = 'a.title_link'

# 서제스트키워드 선택자
daumClickElementCSSQuery = '#q.tf_keyword'
googleClickElementCSSQuery = '#searchform textarea.gLFyf'
youtubeClickElementCSSQuery = 'input#search'

daumDataCSSQuery = "span.txt_query"
googleDataCSSQuery = 'div.wM6W7d span'
youtubeDataCSSQuery = '#sbse0 > div.sbqs_c > b'


getGoogleOpenKeywordCSSQuery = 'div.s75CSd.u60jwe.r2fjmd.AB4Wff'
# 다음광고 선택자
DAUM_AD_normal = '.list_ad' #와이드광고, 프리미엄광고
DAUM_AD_special = '#splinkColl .list_info.mg_cont.clear' # 스페셜광고
DAUM_REL_KEYWORDS = '#netizen_lists_top span' # 관련키워드

# 검색량 선택자
# config(countGoogleSearchVolume)
volumeCSSQuery = 'div#result-stats'

# 순서 선택자
# config(checkGoogleSectionOrder)
rankCSSQuery = 'div#rso.dURPMd > div.MjjYud a'

In [17]:
# 디버그모드 설정 :
debug = True
# 데이터베이스 생성
keywords = {}
# depth 설정
maxDepth = 4
# 작성주제 생성
targets = ['호텔 컨시어지']
# 2차 검색 주제 설정
subjects = [" "]
# subjects = [
# '창업자',
# '브랜드 스토리',
# '탄생',
# # '광고',
# '철학',
# '논란',
# # '브랜딩',
# # '가치',
# # '의미',
# '원칙',
# '역사'
# ]
# subjects = []
limit = 3

In [18]:

driver = webdriver.Chrome()

for target in targets :    
    # alreadyServeyed = []
    # if debug : print('Now Target is : ',target)
    keywords[target] = {'totalKeywords' : {}}
    subjects = [target+' '+subject for subject in subjects]      
    searchKeywords = ''
    # if debug : print('TotalSubjects is : ', subjects)
    for subject in subjects :
        keywords[target][subject] = searchKeywordFormat.copy()
        # if debug : print('Now Subject is : ', subject)
        for depth in range(maxDepth) :
            # if debug : print('now Depth is : ', depth)
            if depth == 0 :
                # if debug : print('this is first trial')                
                searchKeywords = [subject].copy()                
                # if debug : print('*******all keywords we search is ', searchKeywords)
            elif depth > 0 :
                BefSearchKeywords = searchKeywords.copy()                
                searchKeywords = list(keywords[target][subject].keys())
                try :
                    for w in BefSearchKeywords :
                        if w in searchKeywords :
                            searchKeywords = searchKeywords.remove(w)
                except Exception as e :
                    if debug : print(e)    
                # if debug : print('*******all keywords we search is ', searchKeywords)
            
            # searchKeywords = ['파타고니아', '파타고니아+창업주', '파타고니아+창업스토리', '파타고니아+핵심', '파타고니아+명언']
            # searchKeywords = [word.replace(' ', '+') for word in searchKeywords] # 뛰어쓰기 처치            
            # if debug : display(pd.DataFrame(keywords[target]))
            # 키워드 분석 시작
            for searchKeyword in searchKeywords : # 주제어에 대한 반복문                
                if debug : print('NOW PROCESSING THIS (target, searchKeyword, depth): ',target,searchKeyword,depth)

                # 데이터 수집 - 뷰티풀스프
                
                daumOpenKeywords = getOpenKeywords(createDaumURL(searchKeyword), daumOpenKeywordsTag)
                daumOpenKeywords = limitLength(daumOpenKeywords, limit)
                if debug : print('Collecting naverOpenKeywords..')
                # naverOpenKeywords = getOpenKeywords(createNaverURL(searchKeyword), naverOpenKeywordsTag)
                # if debug : print('daum open keywords complete')

                # 데이터수집 - 셀레니움
                
                try:
                    # 1. 대상 웹사이트 접속
                    driver.get(createDaumURL(searchKeyword))
                    daumSuggestKeywords = seleniumClickAndCrawling(driver, daumClickElementCSSQuery, daumDataCSSQuery)
                    daumSuggestKeywords = limitLength(daumSuggestKeywords, limit)
                    # if debug : print('daum suggest keywords complete')
                    driver.get(createGoogleURL(searchKeyword))
                    googleSuggestKeywords = seleniumClickAndCrawling(driver, googleClickElementCSSQuery, googleDataCSSQuery)
                    googleSuggestKeywords = limitLength(googleSuggestKeywords, limit)
                    # if debug : print('google suggest keywords complete')
                    # googleSuggestKeywords = [searchKeyword+" "+word for word in googleSuggestKeywords] # 구글 특수처리                    
                    # if debug : print('google suggest keywords modifying complete')
                    googleOpenKeywords = getGoogleOpenKeywords(driver,searchKeyword, getGoogleOpenKeywordCSSQuery)
                    googleOpenKeywords = limitLength(googleOpenKeywords, limit)
                    # if debug : print('google open keywords complete')
                    
                  
                    
                    # 에러 핸들링
                except Exception as e:
                    if debug : print('[error occured]', e)
                    continue
                # 셀레니움 종료
                # finally:
                #     driver.quit()
                datas = [daumOpenKeywords,
                        daumSuggestKeywords,
                        googleSuggestKeywords,
                        googleOpenKeywords,
                        #  youtubeSuggestKeywords
                        ] #네이버빠짐
                
                merged_dict = {}
                for provider,data in zip(providers,datas) :            
                    # alreadyServeyed += data
                    submittedkeywordValuationFormat = keywordValuationFormat.copy()
                    submittedkeywordValuationFormat['provider'] = provider
                    submittedkeywordValuationFormat['depth'] = depth
                    # if debug : print('modifying format complete')
                    dictionaryTypeData = keywordFormatting(data, submittedkeywordValuationFormat)
                    # if debug : print('apply format complete (list -> dict)')
                    if dictionaryTypeData :
                        merged_dict.update(dictionaryTypeData)
                    # if debug : print('creating merged_dict & saving complete')
                # if debug : print('this is completed merged_Dict', merged_dict)
                keywords[target][subject].update(merged_dict)
                # if debug : display(pd.DataFrame(keywords[target]))
           
                # 광고갯수 측정 : beautifulsoup                
                for key, value in keywords[target][subject].items() :
                    # if key not in alreadyServeyed :
                        count = countDaumADs(createDaumURL(key), key, [DAUM_AD_special,DAUM_AD_normal])
                        # if debug : print('ad counting complete. number of ad :', count, 'at :', key)
                        keywords[target][subject][key]['countDaumADs'] = count
                    
                
                #검색게시물 수랑 순서            
                # driver = webdriver.Chrome()
                for key, value in keywords[target][subject].items() :
                    try :
                        # if key not in alreadyServeyed :
                            time.sleep(2)
                            driver.get(createGoogleURL(key)) # 디렉션
                            volume = countGoogleSearchVolume(driver,volumeCSSQuery)    # 검색량 측정
                            # if debug : print('counting search volume complete volume : ', volume, 'at : ',key)
                            rank = checkGoogleSectionOrder(driver, rankCSSQuery) # 랭크 측정                
                            # if debug : print('checking tistory rank complete. rank : ', rank, 'at : ',key)
                            keywords[target][subject][key]['searchVolume'] = volume # 검색량 저장
                            keywords[target][subject][key]['tistoryRank'] = rank # 랭크 저장
                            # if debug : print('rank and volume saving complete')
                    except Exception as e:
                        if debug : print('[[error occured]]', e)
                
                
                
                keywords[target]['totalKeywords'].update(keywords[target][subject])
                # if debug : print('merge to totalKeywords compelte')
                # if debug : display(pd.DataFrame(keywords[target]))
    

driver.quit()


Now Target is :  호텔 컨시어지
TotalSubjects is :  ['호텔 컨시어지  ']
Now Subject is :  호텔 컨시어지  
now Depth is :  0
this is first trial
*******all keywords we search is  ['호텔 컨시어지  ']


,totalKeywords,호텔 컨시어지


NOW PROCESSING THIS (target, searchKeyword, depth):  호텔 컨시어지 호텔 컨시어지   0
from https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=호텔+컨시어지++
getted openKeywords length 10
걸렀습니다.
daum open keywords complete
click okay
getted suggest keywords 2
daum suggest keywords complete
click okay
getted suggest keywords 13
걸렀습니다.
google suggest keywords complete
google suggest keywords modifying complete
걸렀습니다.
google open keywords complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
this is completed merged_Dict {'호텔 컨시어지 서비스': {'searchVolume': '', 'countDaumADs': '', 'tistoryRank': '', 'provider': 'googleOpenKeywords', 'depth': 0}

,totalKeywords,호텔 컨시어지
호텔 컨시어지 서비스,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
호텔 프론트 업무,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
비즈니스 호텔,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
호텔 컨시어지 연봉,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
호텔 컨시어지 뜻,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."


ad counting complete. number of ad : 0 at : 호텔 컨시어지 서비스
ad counting complete. number of ad : 1 at : 호텔 프론트 업무
ad counting complete. number of ad : 13 at : 비즈니스 호텔
ad counting complete. number of ad : 0 at : 호텔 컨시어지 연봉
ad counting complete. number of ad : 0 at : 호텔 컨시어지 뜻
counting search volume complete volume :  1110000 at :  호텔 컨시어지 서비스
checking tistory rank complete. rank :  2 at :  호텔 컨시어지 서비스
rank and volume saving complete
counting search volume complete volume :  539000 at :  호텔 프론트 업무
checking tistory rank complete. rank :  1 at :  호텔 프론트 업무
rank and volume saving complete
counting search volume complete volume :  11500000 at :  비즈니스 호텔
checking tistory rank complete. rank :  11 at :  비즈니스 호텔
rank and volume saving complete
counting search volume complete volume :  24300 at :  호텔 컨시어지 연봉
checking tistory rank complete. rank :  8 at :  호텔 컨시어지 연봉
rank and volume saving complete
counting search volume complete volume :  90700 at :  호텔 컨시어지 뜻
checking tistory rank complete. rank : 

...

,totalKeywords,호텔 컨시어지
호텔 컨시어지 서비스,"{'searchVolume': 1110000, 'countDaumADs': 0, '...","{'searchVolume': '', 'countDaumADs': '', 'tist..."
호텔 프론트 업무,"{'searchVolume': 539000, 'countDaumADs': 3, 't...","{'searchVolume': 539000, 'countDaumADs': 3, 't..."
비즈니스 호텔,"{'searchVolume': 11500000, 'countDaumADs': 13,...","{'searchVolume': '', 'countDaumADs': '', 'tist..."
호텔 컨시어지 연봉,"{'searchVolume': 24300, 'countDaumADs': 0, 'ti...","{'searchVolume': '', 'countDaumADs': '', 'tist..."
호텔 컨시어지 뜻,"{'searchVolume': 90700, 'countDaumADs': 0, 'ti...","{'searchVolume': 90700, 'countDaumADs': 0, 'ti..."
객실정리 호텔 서비스,"{'searchVolume': 1400000, 'countDaumADs': 4, '...","{'searchVolume': 1400000, 'countDaumADs': 4, '..."
호텔 객실 정리,"{'searchVolume': 1860000, 'countDaumADs': 9, '...","{'searchVolume': 1860000, 'countDaumADs': 9, '..."
컨시어지 서비스 사례,"{'searchVolume': 144000, 'countDaumADs': 1, 't...","{'searchVolume': 144000, 'countDaumADs': 1, 't..."
컨시어지 서비스 뜻,"{'searchVolume': 109000, 'countDaumADs': 3, 't...","{'searchVolume': 109000, 'countDaumADs': 3, 't..."
컨시어지 뜻,"{'searchVolume': 145000, 'countDaumADs': 3, 't...","{'searchVolume': 145000, 'countDaumADs': 3, 't..."


ad counting complete. number of ad : 0 at : 호텔 컨시어지 서비스
ad counting complete. number of ad : 3 at : 호텔 프론트 업무
ad counting complete. number of ad : 13 at : 비즈니스 호텔
ad counting complete. number of ad : 0 at : 호텔 컨시어지 연봉
ad counting complete. number of ad : 0 at : 호텔 컨시어지 뜻
ad counting complete. number of ad : 4 at : 객실정리 호텔 서비스
ad counting complete. number of ad : 9 at : 호텔 객실 정리
ad counting complete. number of ad : 1 at : 컨시어지 서비스 사례
ad counting complete. number of ad : 3 at : 컨시어지 서비스 뜻
ad counting complete. number of ad : 3 at : 컨시어지 뜻
ad counting complete. number of ad : 0 at : 호텔 프런트 업무
ad counting complete. number of ad : 8 at : 호텔 체크아웃
ad counting complete. number of ad : 0 at : 모텔 프론트 업무
ad counting complete. number of ad : 1 at : 호텔 프론트 후기
ad counting complete. number of ad : 1 at : 호텔 프론트 현실
ad counting complete. number of ad : 3 at : 호텔 프론트 알바
ad counting complete. number of ad : 1 at : 호텔 프론트 외모
ad counting complete. number of ad : 12 at : 서울 비즈니스호텔
ad counting complete. numb

,totalKeywords,호텔 컨시어지
호텔 컨시어지 서비스,"{'searchVolume': 1110000, 'countDaumADs': 0, '...","{'searchVolume': 1110000, 'countDaumADs': 0, '..."
호텔 프론트 업무,"{'searchVolume': 539000, 'countDaumADs': 3, 't...","{'searchVolume': 539000, 'countDaumADs': 3, 't..."
비즈니스 호텔,"{'searchVolume': 11500000, 'countDaumADs': 13,...","{'searchVolume': 11500000, 'countDaumADs': 13,..."
호텔 컨시어지 연봉,"{'searchVolume': 24300, 'countDaumADs': 0, 'ti...","{'searchVolume': 24300, 'countDaumADs': 0, 'ti..."
호텔 컨시어지 뜻,"{'searchVolume': 90700, 'countDaumADs': 0, 'ti...","{'searchVolume': 90700, 'countDaumADs': 0, 'ti..."
객실정리 호텔 서비스,"{'searchVolume': 1400000, 'countDaumADs': 4, '...","{'searchVolume': 1400000, 'countDaumADs': 4, '..."
호텔 객실 정리,"{'searchVolume': 1860000, 'countDaumADs': 9, '...","{'searchVolume': 1860000, 'countDaumADs': 9, '..."
컨시어지 서비스 사례,"{'searchVolume': 144000, 'countDaumADs': 1, 't...","{'searchVolume': 144000, 'countDaumADs': 1, 't..."
컨시어지 서비스 뜻,"{'searchVolume': 109000, 'countDaumADs': 3, 't...","{'searchVolume': 109000, 'countDaumADs': 3, 't..."
컨시어지 뜻,"{'searchVolume': 145000, 'countDaumADs': 3, 't...","{'searchVolume': 145000, 'countDaumADs': 3, 't..."


now Depth is :  2


TypeError: argument of type 'NoneType' is not iterable

In [23]:
pd.DataFrame(keywords['호텔 컨시어지']['totalKeywords']).to_csv('호텔컨시어지.csv', encoding='cp949')